In [1]:
import os
import sys
import logging
FORMAT = "%(filename)s:%(lineno)d %(funcName)s %(levelname)s %(message)s"
logging.basicConfig(format=FORMAT, level=logging.INFO)

import re
import imp
import itertools
import random
import numpy as np
from numpy import array
import pandas as pd
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns


# do this before we have a chance to import openbabel!
import rdkit, rdkit.Chem.rdDistGeom, rdkit.DistanceGeometry

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import rdBase

import py3Dmol

from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy.reaction import Reaction


from multi_molecule import *
from multi_reaction import *

from ase.calculators.morse import * #chosing this calculator for now because it's fast
from ase.calculators.dftb import *
from ase.calculators.lj import *
from ase.calculators.emt import *

from copy import deepcopy

thermo.py:760 loadLibraries INFO Loading thermodynamics library from CBS_QB3_1dHR.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from KlippensteinH2O2.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from primaryThermoLibrary.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:760 loadLibraries INFO Loading thermodynamics library from thermo_DFT_CCSDTF12_BAC.py in /Users/nathan/Code/RMG-database/input/thermo/libraries...
thermo.py:774 loadGroups INFO Loading thermodynamics group database from /Users/nathan/Code/RMG-database/input/thermo/groups...
transport.py:294 loadGroups INFO Loading transport group database from /Users/nathan/Code/RMG-database/input/transport/groups...
database.py:165 loadFamilies INFO Loading the user-specified kinetics families from /Users/nathan/Code/RMG-database/input/kinetics/famil

In [2]:
rxn = Multi_Reaction("CCCCCC+[O]O_[CH2]CCCCC+OO", "H_Abstraction")

reaction.py:300 generateBoundsMatrix INFO Generating bounds matrix for CCCCCC.[O]O
reaction.py:338 bmPreEdit INFO Changing lower limit 2.9 to 2.393074
reaction.py:338 bmPreEdit INFO Changing lower limit 3.65 to 3.26214059416
reaction.py:338 bmPreEdit INFO Changing lower limit 2.520343 to 2.457475
molecule.py:140 rd_embed INFO RDKit failed to embed on attempt 1 of 15
molecule.py:140 rd_embed INFO RDKit failed to embed on attempt 2 of 15


In [3]:
TS = deepcopy(rxn.multi_ts)
TS.view_ts()

In [7]:
# First, making the first generation

possible_dihedrals = np.arange(0, 360, 30)


calc = EMT()
TS.ase_ts.set_calculator(calc)

population_size = 50

population = []

for i in range(population_size):
    mol_copy = TS
    dihedrals = []
    
    for torsion in mol_copy.torsions:
        dihedral = np.random.choice(possible_dihedrals)
        dihedrals.append(dihedral)
        i,j,k,l = torsion.indices
        RHS = torsion.RHS
        
        TS.ase_ts.set_dihedral(a1 = i,
                                a2 = j, 
                                a3 = k, 
                                a4 = l, 
                                angle= float(dihedral), 
                                indices=RHS)
    
    mol_copy.update_ts_from_ase_ts()
    
    
    e = mol_copy.ase_ts.get_potential_energy()
    
    population.append( [e] + dihedrals )
    
df = pd.DataFrame(population)
columns = ["Energy"]
for i in range(len(TS.torsion_list)):
    columns = columns +["Torsion " + str(i)] 
df.columns = columns
df = df.sort("Energy")
df


/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:42: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Energy,Torsion 0,Torsion 1,Torsion 2,Torsion 3,Torsion 4,Torsion 5
24,8.980141,300,270,30,330,0,120
27,8.993775,150,30,60,210,240,120
10,8.993950,330,90,0,120,150,30
9,8.994428,240,90,300,210,270,30
25,8.995472,30,60,120,210,30,330
34,8.996809,300,180,60,90,0,240
40,9.000513,270,330,90,240,150,0
37,9.001599,60,210,330,270,120,240
1,9.003865,240,90,30,120,150,150
8,9.005187,300,120,0,240,180,330


In [8]:
crossover_probability = 0.8
mutation_probability = 0.2
top_selection_percent = 0.333333333
population_size = 50

top_population_size = int(top_selection_percent * float(population_size))
bottom_population_size = int((1-top_selection_percent) * float(population_size))


In [10]:
generations = 50
mol_copy = deepcopy(TS)

for generation in range(generations):
    print "This is the {}th generation".format(generation)
    top_population = df.iloc[:top_population_size,:]
    population = []
    for i in range(population_size):
        mf = random.sample(np.arange(top_population_size), 2)
        m = max(mf)
        f = min(mf)

        print "The `male` is {0} and the `female` is {1}".format(m,f)



        #mol_copy = deepcopy(mol)
        dihedrals = []

        for j, torsion in enumerate(mol_copy.torsions):

            print i

            mutation = random.random()

            crossover = random.random()

            if mutation > mutation_probability:
                if crossover > crossover_probability:
                    dihedral = top_population.iloc[m, j+1]
                    print "The male torsion was chosen. The dihedral is {}".format(dihedral)
                else:
                    dihedral = top_population.iloc[f, j+1]
                    print "The male torsion was chosen. The dihedral is {}".format(dihedral)
            else:
                dihedral = np.random.choice(possible_dihedrals)
                print "MUTATION. The dihedral is {}".format(dihedral)

            print
            dihedrals.append(dihedral)
            i,j,k,l = torsion.indices
            RHS = torsion.RHS

            mol_copy.ase_ts.set_dihedral(a1 = i,
                                a2 = j, 
                                a3 = k, 
                                a4 = l, 
                                angle= float(dihedral), 
                                indices=RHS)

        mol_copy.update_ts_from_ase_ts()


        e = mol_copy.ase_ts.get_potential_energy()

        population.append( [e] + dihedrals )

    df = pd.DataFrame(population)
    columns = ["Energy"]
    for i in range(len(TS.torsion_list)):
        columns = columns +["Torsion " + str(i)] 
    df.columns = columns
    """
    df = top_population.append(new_population)"""
    df = df.sort("Energy")
    print df.iloc[0:5,:]
    print "~~~~~~~~~~~~~~~~~~~~~~~~~~~~"

This is the 0th generation
The `male` is 7 and the `female` is 2
0
The male torsion was chosen. The dihedral is 330.0

1
The male torsion was chosen. The dihedral is 90.0

2
The male torsion was chosen. The dihedral is 0.0

0
MUTATION. The dihedral is 60

0
The male torsion was chosen. The dihedral is 120.0

2
MUTATION. The dihedral is 210

The `male` is 8 and the `female` is 7
1
The male torsion was chosen. The dihedral is 60.0

1
The male torsion was chosen. The dihedral is 210.0

2
MUTATION. The dihedral is 30

0
The male torsion was chosen. The dihedral is 120.0

0
The male torsion was chosen. The dihedral is 150.0

2
The male torsion was chosen. The dihedral is 240.0

The `male` is 8 and the `female` is 0
2
The male torsion was chosen. The dihedral is 300.0

1
The male torsion was chosen. The dihedral is 270.0

2
The male torsion was chosen. The dihedral is 30.0

0
The male torsion was chosen. The dihedral is 330.0

0
The male torsion was chosen. The dihedral is 0.0

2
The male to

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:65: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


      Energy  Torsion 0  Torsion 1  Torsion 2  Torsion 3  Torsion 4  Torsion 5
40  8.975640      240.0       90.0      330.0      210.0        0.0       30.0
2   8.980141      300.0      270.0       30.0      330.0        0.0      120.0
38  8.993870      330.0       90.0        0.0      120.0      150.0        0.0
19  8.995072      240.0       90.0      300.0      240.0      270.0       90.0
9   8.995500      300.0      270.0      300.0      330.0        0.0      270.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~
This is the 1th generation
The `male` is 10 and the `female` is 9
0
MUTATION. The dihedral is 30

1
The male torsion was chosen. The dihedral is 180.0

2
The male torsion was chosen. The dihedral is 60.0

0
The male torsion was chosen. The dihedral is 90.0

0
The male torsion was chosen. The dihedral is 0.0

2
The male torsion was chosen. The dihedral is 240.0

The `male` is 9 and the `female` is 4
1
The male torsion was chosen. The dihedral is 60.0

1
The male torsion was chosen. The dihedra

In [11]:
df

,Energy,Torsion 0,Torsion 1,Torsion 2,Torsion 3,Torsion 4,Torsion 5
40,8.952399,0.0,300.0,330.0,210.0,0.0,0.0
16,8.952399,0.0,300.0,330.0,210.0,0.0,0.0
45,8.952399,0.0,300.0,330.0,210.0,0.0,0.0
46,8.952399,0.0,300.0,330.0,210.0,0.0,0.0
41,8.952399,0.0,300.0,330.0,210.0,0.0,0.0
26,8.952399,0.0,300.0,330.0,210.0,0.0,0.0
23,8.952399,0.0,300.0,330.0,210.0,0.0,0.0
28,8.952399,0.0,300.0,330.0,210.0,0.0,0.0
10,8.952399,0.0,300.0,330.0,210.0,0.0,0.0
11,8.952399,0.0,300.0,330.0,210.0,0.0,0.0


In [14]:
def make_ts_conformer_dataframe(rxn):
    
    TS = deepcopy(rxn.multi_ts)
    calc = EMT()
    TS.ase_ts.set_calculator(calc)
    # Getting the torsion combos
    torsion_list = TS.torsions
    torsion_angles = np.arange(0, 360,30) ### You can change the degree step size
    torsion_combos = list( itertools.combinations_with_replacement( torsion_angles, len(torsion_list)) )
    if len(torsion_list) != 1:
        torsion_combos = list(
            set(
                torsion_combos + 
                list(itertools.combinations_with_replacement( 
                    torsion_angles[::-1], len(torsion_list)
                ))))

        results = []

    # Calculating the potential energy for each conformation
    for combo in torsion_combos:

        geo = zip(torsion_list, combo)
        #print geo
        for torsion in geo:
            tor = torsion[0]
            #print tor
            angle = torsion[1]

            i,j,k,l = tor.indices
            RHS =  tor.RHS
            TS.ase_ts.set_dihedral(a1 = i,
                                a2 = j, 
                                a3 = k, 
                                a4 = l, 
                                angle= float(angle), 
                                indices=RHS)
        TS.update_ts_from_ase_ts()
        results.append(list(combo) + [TS.ase_ts.get_potential_energy()])

    # Creating a dataframe of the results
    df = pd.DataFrame(results)
    columns = ["Energy"]
    for i in range(len(torsion_list)):
        columns = ["Torsion " + str(i)] + columns

    df.columns = columns


    return df.sort("Energy")

In [15]:
make_ts_conformer_dataframe(rxn)

/Users/nathan/anaconda/envs/rmg_env/lib/python2.7/site-packages/ipykernel/__main__.py:50: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,Torsion 5,Torsion 4,Torsion 3,Torsion 2,Torsion 1,Torsion 0,Energy
21795,330,300,300,210,0,0,8.970772
5190,30,60,60,150,330,330,8.972077
10663,330,300,300,210,30,0,8.973294
2611,330,300,300,210,30,30,8.974423
20513,30,60,60,150,300,330,8.975301
15105,330,300,300,210,60,0,8.975468
12188,330,300,300,210,60,30,8.975737
751,30,60,60,150,300,300,8.976015
6454,330,300,300,210,60,60,8.976205
3908,330,300,300,210,90,0,8.976821
